# Accelerate SageMaker-PyTorch FSDP Training of Llama-v2 (or GPT-NeoX) with FP8 on P5 instances.
---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

---

In this notebook, you will learn how to accelerate distributed training of the Hugging Face Transformers GPT-NeoX and Llama-v2 models with FP8 mixed precision enabled on P5 instances. We strongly recommend enabling FP8 whenever possible for enhanced training performance.

You can either launch this notebook from an Amazon SageMaker notebook instance which handles all credentials automatically,
or by running it locally and setting credentials manually.

The notebook is accompanied by the following files:
- `train.py`: The entry point script that'll be passed to the SageMaker PyTorch estimator later in this notebook when launching the training job.
- `arguments.py`: This file has functions for argument parsing (i.e. hyperparameters).
- `checkpoints.py`: This file has functions for saving and loading checkpoints.
- `data_utils`: This file has functions for handling S3 URLs.
- `data`: This directory has scripts for preparing and loading data.
- `fsdp_utils.py`: This file has util functions for fully sharded data parallelism.
- `learning_rates.py`: This file has functions for learning rate schedule.
- `logging_utils.py`: This file has functions to handle logging.
- `memory_tracker.py`: This file has functions to track memory usage.
- `requirements.txt`: This file installs the dependencies, including HuggingFace transformers.
- `train_lib.py`: This file has functions for running an end-to-end training of the GPT-NeoX or Llama-v2 model with SMP FSDP, settings for hybrid sharding applied, and implemented with code lines to save, load, and fine-tune the model.
- `train_utils.py`: This file has utility functions for training.

## Additional Resources
- To learn more about launching a multi-node distributed PyTorch training job, see [Launching a Distributed Training Job](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#launching-a-distributed-training-job).
- To learn more about using the SageMaker Python SDK with PyTorch, see [Using PyTorch with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html).
- To learn more about launching a training job in Amazon SageMaker with your own training image, see [Use Your Own Training Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html).

## Prerequisites
You need to create an `S3` bucket to store the input data for training.
This bucket must be located in the same AWS Region that you choose to launch your training job. To learn how to create a `S3` bucket,
see [Create your first S3 bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/creating-bucket.html) in the Amazon S3 documentation.

## Launching Environment

### Amazon SageMaker Notebook
You can run the notebook with an Amazon SageMaker notebook instance without manually setting your aws credentials.

1. Create a new SageMaker notebook instance and open it.
2. Zip the contents of this folder & upload to the instance with the `Upload` button on the top-right.
3. Open a new terminal with `New -> Terminal`.
4. Within the terminal, enter the correct directory and unzip the file.
    1. `cd SageMaker && unzip <your-zip-name-here>.zip`

### Locally
You can run locally by launching a Jupyter notebook server with `jupyter notebook`.
This requires you to set your aws credentials in the environment manually.
See [Configure the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) for more details.

## Amazon SageMaker Initialization
Run the following cell to import SageMaker modules and retrieve information of your current SageMaker work environment,
such as your AWS account ID, the AWS Region, and the ARN of your Amazon SageMaker execution role.
Upgrade SageMaker SDK to the latest version.

**NOTE:** This step might require a kernel restart.

In [ ]:
%pip install --upgrade "sagemaker>=2.224"
%pip install sagemaker-experiments

### AWS Setup

In [ ]:
%%time
import os

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

role = get_execution_role()
print(f"SageMaker Execution Role: {role}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account: `{account}`.")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region: `{region}`.")

sm_boto_client = boto3.client("sagemaker")
sagemaker_session = sagemaker.session.Session(boto_session=session)

# get default bucket
default_bucket = sagemaker_session.default_bucket()
default_bucket_prefix = sagemaker_session.default_bucket_prefix
default_bucket_prefix_path = ""

# If a default bucket prefix is specified, append it to the s3 path
if default_bucket_prefix:
    default_bucket_prefix_path = f"/{default_bucket_prefix}"

print(f"\nDefault bucket for this session: `{default_bucket}`.")

# FSX setup if you plan to use it.
# - Specify FSx Lustre file system id.
FSX_FILE_SYSTEM_TYPE = "FSxLustre"
FSX_FILE_SYSTEM_ID = "fs-XXXXXXXXXXXXXXXXX"  # Sample: "fs-[a-z0-9]+"

# - Specify the SG and subnet used by the FSX, these are passed to SM Estimator so jobs use this as well.
FSX_SECURITY_GROUP_ID = "sg-XXXXXXXXXXXXXXXXX"  # Sample: "sg-[a-z0-9]+"
FSX_SUBNET_ID = "subnet-XXXXXXXXXXXXXXXXX"  # Sample: "subnet-[a-z0-9]+"

# - Specify directory path for input data on the file system.
# You need to provide normalized and absolute path below.
# Your mount name can be provided by you when creating fsx, or generated automatically.
# You can find this mount_name on the FSX page in console.
# Example of fsx generated mount_name: "3x5lhbmv"
# Example base path: "/3x5lhbmv"
FSX_FILE_SYSTEM_BASE_PATH = "/XXXXXXXX"

## Download and Prepare the GLUE/SST2 Dataset

Here you will download, prepare the GLUE/SST2 dataset and then copy the files to S3.

### Install the Hugging Face Transformers and Datasets libraries

In [ ]:
! pip install -q datasets transformers

In [ ]:
# @title Imports.

import datasets
from datasets import load_dataset, load_from_disk, load_metric
from sagemaker.pytorch import PyTorch
import transformers
import logging

from transformers import AutoTokenizer
from transformers.testing_utils import CaptureLogger

logger = logging.getLogger(__name__)

### Choose Model

Choose to train either the `GPT-NeoX` or `Llama-v2` model.

In [ ]:
model_type = "llama_v2"  # ["gpt_neox", "llama_v2"]

### Load data

This section loads the [GLUE/SST2](https://huggingface.co/datasets/glue/viewer/sst2/train) dataset
and splits it to training and validation datasets.
You can update this section to load any HuggingFace dataset you want.

In [ ]:
# @title Prepare datasets.

hyperparameters = {
    "cache_dir": "tmp",
    "dataset_config_name": "sst2",
    "dataset_name": "glue",
    "do_train": True,
    "do_eval": True,
}

raw_datasets = load_dataset(
    hyperparameters["dataset_name"],
    hyperparameters["dataset_config_name"],
)

# NOTE: Remove existing validation dataset, as it is too small to shard across all ranks.
del raw_datasets["validation"]

if "validation" not in raw_datasets.keys():
    validation_percentage = "10%"

    raw_datasets["validation"] = load_dataset(
        hyperparameters["dataset_name"],
        hyperparameters["dataset_config_name"],
        split=f"train[:{validation_percentage}]",
        cache_dir=hyperparameters["cache_dir"],
    )

    raw_datasets["train"] = load_dataset(
        hyperparameters["dataset_name"],
        hyperparameters["dataset_config_name"],
        split=f"train[{validation_percentage}:]",
        cache_dir=hyperparameters["cache_dir"],
    )

### Load tokenizer
Nearly every NLP task begins with a tokenizer,
which converts your text data into a format (token) that can be processed by the NLP model.

The following cell loads a tokenizer for GPT-NeoX-7B using [AutoTokenizer.from_pretrained()](https://huggingface.co/docs/transformers/v4.19.4/en/autoclass_tutorial#autotokenizer).

In [ ]:
# @title Tokenizer.

tokenizer_kwargs = {
    "cache_dir": hyperparameters["cache_dir"],
}

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b", **tokenizer_kwargs)

### Preprocess data

Set up a function to run the tokenizer and group texts into chunks smaller than the block size.

In [ ]:
# @title Util functions.


def tokenize_function(examples):
    tok_logger = transformers.utils.logging.get_logger("transformers.tokenization_utils_base")

    with CaptureLogger(tok_logger) as cl:
        output = tokenizer(examples[text_column_name])
        # clm input could be much much longer than block_size
        if "Token indices sequence length is longer than the" in cl.out:
            tok_logger.warning(
                "^^^^^^^^^^^^^^^^ Please ignore the warning above - this long input will be chunked into smaller bits before being passed to the model."
            )
    return output


# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
        # Split by chunks of max_len.
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# @title Tokenize datasets.

column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

# since this will be pickled to avoid _LazyModule error in Hasher force logger loading before tokenize_function
tok_logger = transformers.utils.logging.get_logger("transformers.tokenization_utils_base")

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    num_proc=1,
    remove_columns=column_names,
    desc="Running tokenizer on dataset",
)


block_size = tokenizer.model_max_length
if block_size > 2048:
    logger.warning(
        f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
        "Picking 2048 instead. You can change that default value by passing --block_size xxx."
    )
    block_size = 2048
else:
    if args.block_size > tokenizer.model_max_length:
        logger.warning(
            f"The block_size passed ({block_size}) is larger than the maximum length for the model"
            f"({tokenizer.model_max_length}). Using block_size={tokenizer.model_max_length}."
        )
    block_size = min(block_size, tokenizer.model_max_length)

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    #     num_proc=args.preprocessing_num_workers,
    desc=f"Grouping texts in chunks of {block_size}",
)

Set additional hyperparameters and S3 paths for mapping the train and validation datasets properly depending on the phase (training or validation) of the training job in each epoch.

In [ ]:
# @title Prepare datasets.

if hyperparameters["do_train"]:
    train_dataset = lm_datasets["train"]
    train_dataset.to_json("./training.json")
    training_dataset_location = f"s3://{default_bucket}{default_bucket_prefix_path}/dataset/train/"

    command = f"aws s3 cp ./training.json {training_dataset_location}"
    os.system(command)
else:
    training_dataset_location = None

if hyperparameters["do_eval"]:
    eval_dataset = lm_datasets["validation"]
    eval_dataset.to_json("./validation.json")
    validation_dataset_location = (
        f"s3://{default_bucket}{default_bucket_prefix_path}/dataset/validation/"
    )

    command = f"aws s3 cp ./validation.json {validation_dataset_location}"
    os.system(command)
else:
    validation_dataset_location = None


if hyperparameters["do_train"]:
    command = "rm ./training.json"
    os.system(command)

if hyperparameters["do_eval"]:
    command = "rm ./validation.json"
    os.system(command)

In [ ]:
%store training_dataset_location
%store validation_dataset_location

%store

## Specify Amazon S3 Bucket Paths
Here you need to specify the paths for training data to be used by your job. The bucket used must be in the same region as where training will run. In the cells above you downloaded the GLUE/SST2 training and validation split datasets and uploaded the json files in an S3 bucket in your account. This example will train on those json files.

After you successfully run this example tensor parallel + fully sharded data parallel training job, you can modify the S3 bucket to where your own dataset is stored.

In [ ]:
%store -r training_dataset_location
%store -r validation_dataset_location

# if you're bringing your own data, uncomment the following lines and specify the locations there
# training_dataset_location = YOUR_S3_BUCKET/training
# validation_dataset_location = YOUR_S3_BUCKET/validation

In [ ]:
s3_train_bucket = training_dataset_location
s3_test_bucket = validation_dataset_location

The following S3 bucket will store the output artifacts of the training job. You can modify this as needed.

In [ ]:
s3_output_bucket = f"s3://sagemaker-{region}-{account}/smp-fsdp-tp/{model_type}-outputdir/"

## Define Data Channels for SageMaker Training Using Amazon S3
In this step, define SageMaker training data channels to the S3 buckets.



In [ ]:
# @title Setup if you do not use fsx.

use_fsx = False

if not use_fsx:
    if s3_train_bucket != None:
        train = sagemaker.inputs.TrainingInput(
            s3_train_bucket, distribution="FullyReplicated", s3_data_type="S3Prefix"
        )
        data_channels = {"train": train}
    else:
        data_channels = {"train": mock_data}
    if s3_test_bucket != None:
        test = sagemaker.inputs.TrainingInput(
            s3_test_bucket, distribution="FullyReplicated", s3_data_type="S3Prefix"
        )
        data_channels["test"] = test
    else:
        data_channels["test"] = mock_data
    print(data_channels)

### (Optional) Set Up and Use Amazon FSx for Data Channels and Checkpoints
While the previous option of using Amazon S3 is easier to setup, using an FSx can be beneficial for performance when dealing with large input sizes and large model sizes and is more stable. In general, checkpointing should be done using FSx.

Please see the instructions from [Distributed Training of Mask-RCNN in Amazon SageMaker Using FSx](https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb) to create an FSx Lustre file system and import the dataset from the S3 bucket to your FSx file system. Note that the FSx file system must be created in a private subnet with internet gateway to ensure that training job has access to the internet. For general guidance on setting an FSx Lustre file system as data input channel, see Configure Data Input Channel to Use Amazon FSx for Lustre.

In [ ]:
# @title Setup FSX as needed.

# Instructions obtained from:
# https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb

if use_fsx:
    from sagemaker.inputs import FileSystemInput

    train = FileSystemInput(
        directory_path=FSX_FILE_SYSTEM_BASE_PATH,
        file_system_access_mode="rw",
        file_system_id=FSX_FILE_SYSTEM_ID,
        file_system_type=FSX_FILE_SYSTEM_TYPE,
    )

    data_channels = {"train": train, "test": train}

## Set Hyperparameters, Metric Definitions, and MPI Options

### FP8 Datatype

FP8 (an 8-bit floating point precision) is a datatype suitable for mixed precision training, a natural progression for accelerating deep learning training beyond the 16-bit formats such as FP16 and BF16. NVIDIA H100 GPUs came out supporting mixed precision training with FP8 data types. With P5 instances equipped with H100 GPUs, you can accelerate distributed training with FP8 mixed precision.

For more information, see [FP8 training on P5](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-core-features-v2-fp8-training-on-p5.html).


In [ ]:
# @title Hyperparameters.

fp8 = 1  # Enable FP8 mixed precision. 1=True, 0=False.
tensor_parallel_degree = 2  # An integer in [1, world_size]
hybrid_shard_degree = (
    4  # An integer in [0, world_size // tensor_parallel_degree] and its default value is 0.
)
activation_loading_horizon = (
    2  # Activation loading horizon, a positive integer and its default value is 2.
)
save_steps = 10  # Save step interval.
max_steps = 15  # Maximum training steps.
offload_activations = False  # Activation offloading.

hyperparameters = {
    "activation_checkpointing": 1,
    "auto_wrap_policy": "transformer_auto_wrap_policy",
    "backward_fetch_policy": "backward_pre",
    "beta1": 0.9,
    "beta2": 0.95,
    "bf16": 1,
    "checkpoint_dir": "/opt/ml/checkpoints",
    "checkpoint_freq": save_steps,
    "clean_cache": 0,
    "delayed_param": 1,
    "enable_memory_profiling": 0,
    "epochs": 100,
    "fast_validation": 0,
    "forward_prefetch": 1,
    "fp8": fp8,
    "limit_all_gathers": 1,
    "logging_freq": 1,
    "lr": 0.0001,
    "lr_decay_iters": 47683,
    "lr_decay_style": "cosine",
    "max_steps": max_steps,
    "min_lr": 1e-05,
    "model_type": model_type,
    "num_kept_checkpoints": 2,
    "plateau": 0.0,
    "seed": 12345,
    "sharding_strategy": "hybrid_shard",
    "train_batch_size": 32,
    "use_smp_flash_attn": 1,
    "use_smp_implementation": 1,
    "val_batch_size": 32,
    "validation_freq": save_steps,
    "vocab_size": 50257,
    "warmup": 0.0032,
    "weight_decay": 0.2,
    "zipped_data": 0,
}

if use_fsx:
    # make sure to update paths for training_dir and test_dir based on the paths of datasets in fsx
    # If you want to resume training, set checkpoint_dir to the same path as a previous job.
    SM_TRAIN_DIR = "/opt/ml/input/data/train"
    hyperparameters["checkpoint_dir"] = f"{SM_TRAIN_DIR}/smp-v2/{model_type}/checkpointdir"
    hyperparameters["training_dir"] = (
        f"{SM_TRAIN_DIR}/datasets/pytorch-gpt2-data/pytorch_gpt2/train_synthetic"
    )
    hyperparameters["test_dir"] = (
        f"{SM_TRAIN_DIR}/datasets/pytorch-gpt2-data/pytorch_gpt2/val_synthetic"
    )

# The checkpoint path (hyperparameters['checkpoint_dir'] or checkpoint_s3_uri) is not unique per job.
# You need to modify as needed for different runs.
# If same path is used for unrelated runs, this may increase time when downloading unnecessary checkpoints,
# and cause conflicts when loading checkpoints.

metric_definitions = [
    {"Name": "base_metric", "Regex": "<><><><><><>"}
]  # Add your custom metric definitions

In [ ]:
# @title Specify model config based on model name and model size.

model_size = 7  # 7B or 65B parameters.

model_configs = {
    "gpt_neox": {
        7: {
            "hidden_width": 4096,
            "max_context_width": 2048,
            "num_heads": 32,
            "num_layers": 32,
        },
        65: {
            "hidden_width": 8192,
            "max_context_width": 2048,
            "num_heads": 64,
            "num_layers": 80,
        },
    },
    "llama_v2": {
        7: {
            "hidden_width": 4096,
            "llama_intermediate_size": 11008,
            "max_context_width": 2048,
            "num_heads": 32,
            "num_layers": 32,
        },
        65: {
            "hidden_width": 8192,
            "llama_intermediate_size": 22016,
            "max_context_width": 2048,
            "num_heads": 64,
            "num_layers": 80,
        },
    },
}

model_params = model_configs.get(model_type, {}).get(model_size)
if model_params is None:
    raise RuntimeError(
        f"Unknown model config from (name, size) = ({model_type}, {model_size:02d}B)."
    )

hyperparameters.update(model_params)

## Specify Essential Parameters for a SageMaker Training Job

Next, you use the `SageMaker Estimator class` to define a SageMaker Training Job,
passing values through the following parameters for training job name,
the number of EC2 instances, the instance type,
and the size of the volume attached to the instances.

- `instance_count`
- `instance_type`
- `volume_size`
- `base_job_name`

### Update the Type and Number of EC2 Instance to Use

The instance type and the number of instances you specify to the `instance_type` and `instance_count` parameters,
respectively, determine the total number of GPUs (`world_size`).

$$\text{world_size = (the number of GPUs on a single instance)} \times \text{(the number of instances)}$$

In [ ]:
# @title Setup instances: type and count.

instance_type = "ml.p5.48xlarge"

# You need >= 1 p5 for 7b model.
# You need >= 8 p5 for 65b model.
instance_count = 1

# Set to the number of GPUs on that instance.
processes_per_host = 8

### Specify a Base Job Name

In [ ]:
# @title Job name.

instance_type_str = instance_type.split(".")[1] + instance_type.split(".")[2][:3]

base_job_name = f'smp-{model_size:02d}b-{instance_type_str}-fp8-{fp8}-hs{hybrid_shard_degree}-tp{tensor_parallel_degree}-ao{offload_activations}-bs{hyperparameters["train_batch_size"]:02d}'

print(f"Base job name: `{base_job_name}`.")

In [ ]:
if not use_fsx:
    # If you want to resume training, set checkpoint_s3_uri to the same path as a previous job.
    # Previous checkpoint to load must have same model config.
    checkpoint_bucket = f"s3://sagemaker-{region}-{account}/"
    checkpoint_s3_uri = (
        f"{checkpoint_bucket}/experiments/smp-fsdp-tp-{model_type}-checkpoints/{base_job_name}/"
    )

In [ ]:
# @title Estimator.

if use_fsx:
    # Use the security group and subnet that was used to create the fsx filesystem.
    kwargs = {
        "security_group_ids": [FSX_SECURITY_GROUP_ID],
        "subnets": [FSX_SUBNET_ID],
    }
    print(kwargs)
else:
    kwargs = {}

smp_estimator = PyTorch(
    entry_point="train.py",
    hyperparameters=hyperparameters,
    source_dir=os.path.join(os.getcwd(), "../shared-scripts"),
    role=role,
    checkpoint_s3_uri=checkpoint_s3_uri if not use_fsx else None,
    checkpoint_local_path=hyperparameters["checkpoint_dir"] if use_fsx else None,
    instance_type=instance_type,
    volume_size=400,
    instance_count=instance_count,
    sagemaker_session=sagemaker_session,
    distribution={
        "torch_distributed": {
            "enabled": True,
        },
        "smdistributed": {
            "modelparallel": {
                "enabled": True,
                "parameters": {
                    "activation_loading_horizon": activation_loading_horizon,
                    "hybrid_shard_degree": hybrid_shard_degree,
                    "sm_activation_offloading": offload_activations,
                    "tensor_parallel_degree": tensor_parallel_degree,
                },
            }
        },
    },
    py_version="py311",
    framework_version="2.3.1",
    # image_uri=$IMAGE,  # Either provide `framework_version` or `image_uri`
    output_path=s3_output_bucket,
    max_run=86400,
    debugger_hook_config=False,
    base_job_name=base_job_name,
    metric_definitions=metric_definitions,
    **kwargs,
)

### Start Training

Finally, run the `estimator.fit` method to launch the SageMaker training job of the model with hybrid sharding and activation offloading.

In [ ]:
# @title Start training.

smp_estimator.fit(inputs=data_channels)

## Resume from Checkpoint
You can continue a previous SageMaker training job from a saved checkpoint. Set your checkpoint directory to the appropriate checkpoint you want to resume from and pass in the `checkpoint_s3_uri` from a previous job.

In [ ]:
# @title Resume training.

resume_from_checkpoint = True

if resume_from_checkpoint:
    hyperparameters["resume_from_checkpoint"] = os.path.join(
        hyperparameters["checkpoint_dir"], f"{model_type}-{save_steps}steps"
    )

    smp_estimator = PyTorch(
        entry_point="train.py",
        hyperparameters=hyperparameters,
        source_dir=os.path.join(os.getcwd(), "../shared-scripts"),
        role=role,
        checkpoint_s3_uri=checkpoint_s3_uri if not use_fsx else None,
        checkpoint_local_path=hyperparameters["checkpoint_dir"] if use_fsx else None,
        instance_type=instance_type,
        volume_size=400,
        instance_count=instance_count,
        sagemaker_session=sagemaker_session,
        distribution={
            "torch_distributed": {
                "enabled": True,
            },
            "smdistributed": {
                "modelparallel": {
                    "enabled": True,
                    "parameters": {
                        "activation_loading_horizon": activation_loading_horizon,
                        "hybrid_shard_degree": hybrid_shard_degree,
                        "sm_activation_offloading": offload_activations,
                        "tensor_parallel_degree": tensor_parallel_degree,
                    },
                }
            },
        },
        py_version="py311",
        framework_version="2.3.1",
        # image_uri=$IMAGE,  # Either provide `framework_version` or `image_uri`
        output_path=s3_output_bucket,
        max_run=86400,
        debugger_hook_config=False,
        base_job_name=f"resume-{base_job_name}",
        metric_definitions=metric_definitions,
        **kwargs,
    )

    smp_estimator.fit(inputs=data_channels)

## Access the Launched SM Training Job
You can access the launched training job from [SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html).  
Go to `Amazon SageMaker -> Training -> Training jobs`.  
You can also access the training logs from here with `View Logs` which opens CloudWatch directly.

## Access the Training Logs

You can access the training logs from [Amazon CloudWatch](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/WhatIsCloudWatch.html).

You can use CloudWatch to track SageMaker GPU and memory utilization during training and inference. To view the metrics and logs that SageMaker writes to CloudWatch, see [SageMaker Jobs and Endpoint Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/monitoring-cloudwatch.html#cloudwatch-metrics-jobs) in the Amazon SageMaker Developer Guide.

If you are a new user of CloudWatch, see [Getting Started with Amazon CloudWatch](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/GettingStarted.html).

For additional information on monitoring and analyzing Amazon SageMaker training jobs, see [Monitor and Analyze Training Jobs Using Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html).

## Deploy Trained Model for Inference

In most cases, a trained model can be deployed on a single device for inference because inference only requires a small amount of memory.

After you build and train your models, you can deploy them to get predictions in one of two ways:

* To set up a persistent endpoint to get predictions from your models, use SageMaker hosting services. For an overview on deploying a single model or multiple models with SageMaker hosting services, see [Deploy a Model on SageMaker Hosting Services](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-deployment.html#how-it-works-hosting).
* To get predictions for an entire dataset, use SageMaker batch transform. For an overview on deploying a model with SageMaker Batch Transform, see [Get Inferences for an Entire Dataset with Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html).

To learn more about deploying models for inference using SageMaker, see [Deploy Models for Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).


## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/generative_ai|sm-fsdp_training_of_llama_v2_with_fp8_on_p5.ipynb)
